In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import random

spark = SparkSession.builder \
    .appName("StreamPulse-PlanAudit") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.sql.adaptive.enabled", "false") \
    .config("spark.sql.autoBroadcastJoinThreshold", "-1") \
    .getOrCreate()


In [2]:
random.seed(42)

# Listening events (large table - 500K rows)
events_data = []
for i in range(500000):
    events_data.append((
        f"EVT-{i+1:07d}",
        f"USR-{random.randint(1, 100000):06d}",
        f"TRK-{random.randint(1, 50000):06d}",
        f"ART-{random.randint(1, 5000):05d}",
        random.randint(10, 300),
        random.choice([True, False]),
        random.choice(["mobile", "desktop", "smart_speaker", "tablet"]),
        random.choice(["free", "premium"]),
        f"202{random.randint(3,4)}-{random.randint(1,12):02d}-{random.randint(1,28):02d}",
    ))

events = spark.createDataFrame(events_data,
    ["event_id", "user_id", "track_id", "artist_id", "duration_sec",
     "completed", "device", "tier", "event_date"]) \
    .withColumn("event_date", col("event_date").cast("date")) \
    .withColumn("year", year(col("event_date"))) \
    .withColumn("month", month(col("event_date")))

events.write.parquet("audit_data/events", mode="overwrite", partitionBy=["year"])

# Artists (small table - 5K rows)
artist_data = [(f"ART-{i+1:05d}", f"Artist {i+1}",
                random.choice(["Pop", "Rock", "Hip-Hop", "Jazz", "Electronic"]),
                random.choice(["US", "UK", "KR", "JP", "DE"]))
               for i in range(5000)]
artists = spark.createDataFrame(artist_data, ["artist_id", "name", "genre", "country"])
artists.write.parquet("audit_data/artists", mode="overwrite")

# Tracks (medium table - 50K rows)
track_data = [(f"TRK-{i+1:06d}", f"Track {i+1}",
               f"ART-{random.randint(1, 5000):05d}",
               random.randint(60, 400),
               random.randint(2018, 2024))
              for i in range(50000)]
tracks = spark.createDataFrame(track_data,
    ["track_id", "title", "artist_id", "track_duration", "release_year"])
tracks.write.parquet("audit_data/tracks", mode="overwrite")

# Reload from Parquet
events = spark.read.parquet("audit_data/events")
artists = spark.read.parquet("audit_data/artists")
tracks = spark.read.parquet("audit_data/tracks")

print(f"Events: {events.count()} | Artists: {artists.count()} | Tracks: {tracks.count()}")


Events: 500000 | Artists: 5000 | Tracks: 50000




Part 2
Analyze Query Plans (40 min)
For each query below, run .explain(mode="formatted") and document:

The join strategy used (Broadcast or SortMerge)
Number of Exchange (shuffle) operations
Whether predicate pushdown / partition filters are applied
Columns in ReadSchema (column pruning)
Any whole-stage code generation (* markers)
Your assessment: Efficient / Needs Optimization / Critical Issue


In [4]:
# Query 1: Simple filter and select

q1 = events.filter(col("year") == 2024) \
    .filter(col("completed") == True) \
    .select("event_id", "user_id", "duration_sec")

print("QUERY 1: Simple filter and select")
q1.explain(mode="formatted")

QUERY 1: Simple filter and select
== Physical Plan ==
* Project (4)
+- * Filter (3)
   +- * ColumnarToRow (2)
      +- Scan parquet  (1)


(1) Scan parquet 
Output [5]: [event_id#21, user_id#22, duration_sec#25L, completed#26, year#31]
Batched: true
Location: InMemoryFileIndex [file:/content/audit_data/events]
PartitionFilters: [isnotnull(year#31), (year#31 = 2024)]
PushedFilters: [IsNotNull(completed), EqualTo(completed,true)]
ReadSchema: struct<event_id:string,user_id:string,duration_sec:bigint,completed:boolean>

(2) ColumnarToRow [codegen id : 1]
Input [5]: [event_id#21, user_id#22, duration_sec#25L, completed#26, year#31]

(3) Filter [codegen id : 1]
Input [5]: [event_id#21, user_id#22, duration_sec#25L, completed#26, year#31]
Condition : (isnotnull(completed#26) AND completed#26)

(4) Project [codegen id : 1]
Output [3]: [event_id#21, user_id#22, duration_sec#25L]
Input [5]: [event_id#21, user_id#22, duration_sec#25L, completed#26, year#31]




| Aspect             | Your Finding                    |
| ------------------ | ------------------------------- |
| Scan type          | FileScan parquet                |
| PartitionFilters   | (year = 2024)                   |
| PushedFilters      | (completed = true)              |
| ReadSchema columns | event_id, user_id, duration_sec |
| Exchange count     | 0                               |
| Assessment         | Efficient                       |


In [5]:
# Query 2: Join events with artists


q2 = events.join(artists, "artist_id") \
    .filter(col("year") == 2024) \
    .select("event_id", "name", "genre", "duration_sec")

print("QUERY 2: Events JOIN Artists (filter after join)")
q2.explain(mode="formatted")


QUERY 2: Events JOIN Artists (filter after join)
== Physical Plan ==
* Project (13)
+- * SortMergeJoin Inner (12)
   :- * Sort (6)
   :  +- Exchange (5)
   :     +- * Project (4)
   :        +- * Filter (3)
   :           +- * ColumnarToRow (2)
   :              +- Scan parquet  (1)
   +- * Sort (11)
      +- Exchange (10)
         +- * Filter (9)
            +- * ColumnarToRow (8)
               +- Scan parquet  (7)


(1) Scan parquet 
Output [4]: [event_id#21, artist_id#24, duration_sec#25L, year#31]
Batched: true
Location: InMemoryFileIndex [file:/content/audit_data/events]
PartitionFilters: [isnotnull(year#31), (year#31 = 2024)]
PushedFilters: [IsNotNull(artist_id)]
ReadSchema: struct<event_id:string,artist_id:string,duration_sec:bigint>

(2) ColumnarToRow [codegen id : 1]
Input [4]: [event_id#21, artist_id#24, duration_sec#25L, year#31]

(3) Filter [codegen id : 1]
Input [4]: [event_id#21, artist_id#24, duration_sec#25L, year#31]
Condition : isnotnull(artist_id#24)

(4) Project [c

| Aspect             | Your Finding                    |
| ------------------ | ------------------------------- |
| Join strategy      | SortMergeJoin                   |
| Artists table size | ~5K rows (small!)               |
| Exchange count     | 2                               |
| Could broadcast?   | Yes                             |
| Filter placement   | Applied after join (suboptimal) |
| Assessment         | Needs Optimization              |


In [6]:
q3 = events.join(tracks, "track_id") \
    .join(artists, "artist_id") \
    .filter(col("year") == 2024) \
    .filter(col("genre") == "Pop") \
    .groupBy("name") \
    .agg(count("*").alias("play_count"), avg("duration_sec").alias("avg_duration"))

print("QUERY 3: Three-table join with aggregation")
q3.explain(mode="formatted")


QUERY 3: Three-table join with aggregation
== Physical Plan ==
* HashAggregate (26)
+- Exchange (25)
   +- * HashAggregate (24)
      +- * Project (23)
         +- * SortMergeJoin Inner (22)
            :- * Sort (15)
            :  +- Exchange (14)
            :     +- * Project (13)
            :        +- * SortMergeJoin Inner (12)
            :           :- * Sort (6)
            :           :  +- Exchange (5)
            :           :     +- * Project (4)
            :           :        +- * Filter (3)
            :           :           +- * ColumnarToRow (2)
            :           :              +- Scan parquet  (1)
            :           +- * Sort (11)
            :              +- Exchange (10)
            :                 +- * Filter (9)
            :                    +- * ColumnarToRow (8)
            :                       +- Scan parquet  (7)
            +- * Sort (21)
               +- Exchange (20)
                  +- * Project (19)
                     +- * Filt

| Aspect               | Your Finding                                |
| -------------------- | ------------------------------------------- |
| Join 1 strategy      | SortMergeJoin (events ↔ tracks)             |
| Join 2 strategy      | SortMergeJoin (result ↔ artists)            |
| Total Exchange count | 4–5 (two joins + aggregation shuffle)       |
| Filter on year?      | Yes (pushed to partition pruning)           |
| Filter on genre?     | No (applied after join, not pushed to scan) |
| Assessment           | Critical Issue (excessive shuffle)          |



In [7]:
# Query 4: Aggregation with multiple actions (simulated)

enriched = events.join(artists, "artist_id").filter(col("year") == 2024)

print("QUERY 4a: Genre aggregation")
q4a = enriched.groupBy("genre").agg(count("*").alias("plays"))
q4a.explain(mode="formatted")

print("\nQUERY 4b: Device aggregation (same enriched source)")
q4b = enriched.groupBy("device").agg(avg("duration_sec").alias("avg_dur"))
q4b.explain(mode="formatted")


QUERY 4a: Genre aggregation
== Physical Plan ==
* HashAggregate (16)
+- Exchange (15)
   +- * HashAggregate (14)
      +- * Project (13)
         +- * SortMergeJoin Inner (12)
            :- * Sort (6)
            :  +- Exchange (5)
            :     +- * Project (4)
            :        +- * Filter (3)
            :           +- * ColumnarToRow (2)
            :              +- Scan parquet  (1)
            +- * Sort (11)
               +- Exchange (10)
                  +- * Filter (9)
                     +- * ColumnarToRow (8)
                        +- Scan parquet  (7)


(1) Scan parquet 
Output [2]: [artist_id#24, year#31]
Batched: true
Location: InMemoryFileIndex [file:/content/audit_data/events]
PartitionFilters: [isnotnull(year#31), (year#31 = 2024)]
PushedFilters: [IsNotNull(artist_id)]
ReadSchema: struct<artist_id:string>

(2) ColumnarToRow [codegen id : 1]
Input [2]: [artist_id#24, year#31]

(3) Filter [codegen id : 1]
Input [2]: [artist_id#24, year#31]
Condition : isnotnu

| Aspect                            | Your Finding                                |
| --------------------------------- | ------------------------------------------- |
| Does 4a and 4b share computation? | No                                          |
| Is enriched cached?               | No                                          |
| Redundant work                    | Join executed twice                         |
| Assessment                        | Needs Optimization (recomputation overhead) |


In [8]:
# Query 5: Self-reference (events aggregated then joined back) - Self-join pattern

popular = events.groupBy("track_id").agg(count("*").alias("play_count")) \
    .filter(col("play_count") > 10)

q5 = events.join(popular, "track_id") \
    .select("event_id", "user_id", "track_id", "play_count")

print("QUERY 5: Self-reference (events aggregated then joined back)")
q5.explain(mode="formatted")


QUERY 5: Self-reference (events aggregated then joined back)
== Physical Plan ==
* Project (17)
+- * SortMergeJoin Inner (16)
   :- * Sort (6)
   :  +- Exchange (5)
   :     +- * Project (4)
   :        +- * Filter (3)
   :           +- * ColumnarToRow (2)
   :              +- Scan parquet  (1)
   +- * Sort (15)
      +- * Filter (14)
         +- * HashAggregate (13)
            +- Exchange (12)
               +- * HashAggregate (11)
                  +- * Project (10)
                     +- * Filter (9)
                        +- * ColumnarToRow (8)
                           +- Scan parquet  (7)


(1) Scan parquet 
Output [4]: [event_id#21, user_id#22, track_id#23, year#31]
Batched: true
Location: InMemoryFileIndex [file:/content/audit_data/events]
PushedFilters: [IsNotNull(track_id)]
ReadSchema: struct<event_id:string,user_id:string,track_id:string>

(2) ColumnarToRow [codegen id : 1]
Input [4]: [event_id#21, user_id#22, track_id#23, year#31]

(3) Filter [codegen id : 1]
Input [4]:

| Aspect                            | Your Finding                             |
| --------------------------------- | ---------------------------------------- |
| How many times is events scanned? | 2                                        |
| Exchange count                    | 2–3 (aggregation shuffle + join shuffle) |
| Join strategy                     | SortMergeJoin                            |
| Could caching help?               | Yes                                      |
| Assessment                        | Needs Optimization                       |



# Part 3: Plan Analysis Repor

StreamPulse Execution Plan Audit Report
Summary

Total queries analyzed: 5

Queries needing optimization: 4

Most common issue: Excessive shuffle due to disabled broadcast joins

Estimated total improvement potential: 30–60% reduction in shuffle cost

Query-by-Query Findings
Query 1: Simple Filter

Status: Efficient

Issues found: None

Recommendation: No changes required

Query 2: Events-Artists Join

Status: Needs Optimization

Issues found:

SortMergeJoin used for small dimension table

Unnecessary shuffle

Filter applied after join

Recommendation:

Apply filter before join

Use broadcast join

Query 3: Three-table Join

Status: Critical Issue

Issues found:

Two SortMergeJoins

Multiple shuffle exchanges

Genre filter not pushed to scan

Recommendation:

Broadcast artists

Filter year before joins

Consider broadcasting tracks if feasible

Query 4: Multiple Aggregations

Status: Needs Optimization

Issues found:

Join recomputed twice

No caching strategy

Recommendation:

Cache enriched dataframe

Query 5: Self Join Pattern

Status: Needs Optimization

Issues found:

events scanned twice

Shuffle-heavy aggregation + join

Recommendation:

Cache popular dataframe

Consider broadcast if result small

Priority Recommendations

Enable broadcast joins for small dimension tables

Filter early before joins

Cache reused intermediate datasets

Configuration Recommendations

spark.sql.autoBroadcastJoinThreshold: Set to default (10MB)

spark.sql.shuffle.partitions: Reduce from 200 to 100

Caching strategy: Cache reused joined datasets and small aggregations

In [9]:
# Part 4: Optimization Proposals

# query 2
# PROPOSED: Enable broadcast for small artists table
q2_optimized = events \
    .filter(col("year") == 2024) \
    .join(broadcast(artists), "artist_id") \
    .select("event_id", "name", "genre", "duration_sec")


In [10]:
# Query 3
q3_optimized = events \
    .filter(col("year") == 2024) \
    .join(tracks, "track_id") \
    .join(broadcast(artists), "artist_id") \
    .filter(col("genre") == "Pop") \
    .groupBy("name") \
    .agg(count("*").alias("play_count"),
         avg("duration_sec").alias("avg_duration"))

In [11]:
# Query 4
enriched = events \
    .filter(col("year") == 2024) \
    .join(broadcast(artists), "artist_id") \
    .cache()

q4a = enriched.groupBy("genre").agg(count("*").alias("plays"))
q4b = enriched.groupBy("device").agg(avg("duration_sec").alias("avg_dur"))

In [12]:
# Query 5
popular = events.groupBy("track_id") \
    .agg(count("*").alias("play_count")) \
    .filter(col("play_count") > 10) \
    .cache()

q5_optimized = events.join(broadcast(popular), "track_id") \
    .select("event_id", "user_id", "track_id", "play_count")

Priority Recommendations

Enable broadcast joins for small dimension tables to eliminate unnecessary shuffle operations (highest impact).

Apply partition filters before joins to reduce dataset size prior to shuffle.

Cache reused intermediate datasets to prevent recomputation in multi-action queries.

Tune shuffle partition configuration to better match cluster size